In [4]:
from py2neo import Graph, Node, Relationship, NodeMatcher

class KnowledgeGraph:
    def __init__(self, uri="neo4j+s://d5800584.databases.neo4j.io", user="neo4j", password="wPQh0QgH3kzqb8iBSAenCL54ZX7suzKy4vt4_jdj6tU"):
        self.graph = Graph(uri, auth=(user, password))
        self.matcher = NodeMatcher(self.graph)

    # Clear the entire graph (optional)
    def clear_graph(self):
        self.graph.run("MATCH (n) DETACH DELETE n")
        print("Graph cleared.")

    # Batch add multiple users
    def add_users(self, user_names):
        tx = self.graph.begin()
        for name in user_names:
            user = Node("User", name=name)
            tx.merge(user, "User", "name")  # Ensures no duplicates
            print(f"User '{name}' added.")
        tx.commit()

    # Batch add multiple books
    def add_books(self, books):
        tx = self.graph.begin()
        for title in books:
            book = Node("Book", title=title)
            tx.merge(book, "Book", "title")
            print(f"Book '{title}' added.")
        tx.commit()

    # Batch add multiple authors
    def add_authors(self, authors):
        tx = self.graph.begin()
        for name in authors:
            author = Node("Author", name=name)
            tx.merge(author, "Author", "name")
            print(f"Author '{name}' added.")
        tx.commit()

    # Batch add multiple genres
    def add_genres(self, genres):
        tx = self.graph.begin()
        for genre in genres:
            genre_node = Node("Genre", name=genre)
            tx.merge(genre_node, "Genre", "name")
            print(f"Genre '{genre}' added.")
        tx.commit()

    # Add relationships between users and books they like
    def add_likes_relationship(self, user_name, book_title):
        user = self.matcher.match("User", name=user_name).first()
        book = self.matcher.match("Book", title=book_title).first()
        if user and book:
            rel = Relationship(user, "LIKES", book)
            self.graph.merge(rel)
            print(f"User '{user_name}' likes the book '{book_title}'.")
        else:
            print(f"User '{user_name}' or Book '{book_title}' not found!")

    # Add relationships between authors and the books they wrote
    def add_wrote_relationship(self, author_name, book_title):
        author = self.matcher.match("Author", name=author_name).first()
        book = self.matcher.match("Book", title=book_title).first()
        if author and book:
            rel = Relationship(author, "WROTE", book)
            self.graph.merge(rel)
            print(f"Author '{author_name}' wrote the book '{book_title}'.")
        else:
            print(f"Author '{author_name}' or Book '{book_title}' not found!")

    # Add relationships between books and genres they belong to
    def add_belongs_to_relationship(self, book_title, genre_name):
        book = self.matcher.match("Book", title=book_title).first()
        genre = self.matcher.match("Genre", name=genre_name).first()
        if book and genre:
            rel = Relationship(book, "BELONGS_TO", genre)
            self.graph.merge(rel)
            print(f"Book '{book_title}' belongs to genre '{genre_name}'.")
        else:
            print(f"Book '{book_title}' or Genre '{genre_name}' not found!")

    # Query: Find books liked by a specific user
    def find_books_liked_by_user(self, user_name):
        query = """
        MATCH (u:User)-[:LIKES]->(b:Book)
        WHERE u.name = $user_name
        RETURN b.title AS title
        """
        results = self.graph.run(query, user_name=user_name).data()
        return [record['title'] for record in results]

    # Query: Recommend books based on genre preferences
    def recommend_books_by_genre(self, user_name):
        query = """
        MATCH (u:User)-[:LIKES]->(:Book)-[:BELONGS_TO]->(g:Genre)<-[:BELONGS_TO]-(b:Book)
        WHERE u.name = $user_name
        RETURN DISTINCT b.title AS title
        LIMIT 5
        """
        results = self.graph.run(query, user_name=user_name).data()
        return [record['title'] for record in results]

    # Query: Recommend books based on authors a user likes
    def recommend_books_by_author(self, user_name):
        query = """
        MATCH (u:User)-[:LIKES]->(:Book)<-[:WROTE]-(a:Author)-[:WROTE]->(b:Book)
        WHERE u.name = $user_name
        RETURN DISTINCT b.title AS title
        LIMIT 5
        """
        results = self.graph.run(query, user_name=user_name).data()
        return [record['title'] for record in results]


# Example usage
if __name__ == "__main__":
    kg = KnowledgeGraph()

    # Clear the graph (optional)
    kg.clear_graph()

    # Add Users, Books, Authors, and Genres
    users = ["Alice", "Bob", "Carol"]
    books = ["The Catcher in the Rye", "1984", "To Kill a Mockingbird", "Brave New World"]
    authors = ["J.D. Salinger", "George Orwell", "Harper Lee", "Aldous Huxley"]
    genres = ["Literature", "Dystopian", "Classic"]

    kg.add_users(users)
    kg.add_books(books)
    kg.add_authors(authors)
    kg.add_genres(genres)

    # Add relationships
    kg.add_likes_relationship("Alice", "The Catcher in the Rye")
    kg.add_likes_relationship("Bob", "1984")
    kg.add_likes_relationship("Carol", "To Kill a Mockingbird")

    kg.add_wrote_relationship("J.D. Salinger", "The Catcher in the Rye")
    kg.add_wrote_relationship("George Orwell", "1984")
    kg.add_wrote_relationship("Harper Lee", "To Kill a Mockingbird")
    kg.add_wrote_relationship("Aldous Huxley", "Brave New World")

    kg.add_belongs_to_relationship("The Catcher in the Rye", "Literature")
    kg.add_belongs_to_relationship("1984", "Dystopian")
    kg.add_belongs_to_relationship("To Kill a Mockingbird", "Classic")
    kg.add_belongs_to_relationship("Brave New World", "Dystopian")

    # Queries
    alice_likes = kg.find_books_liked_by_user("Alice")
    print("Books liked by Alice:", alice_likes)

    recommendations_by_genre = kg.recommend_books_by_genre("Alice")
    print("Books recommended for Alice (by genre):", recommendations_by_genre)

    recommendations_by_author = kg.recommend_books_by_author("Alice")
    print("Books recommended for Alice (by author):", recommendations_by_author)


Graph cleared.
User 'Alice' added.
User 'Bob' added.
User 'Carol' added.
Book 'The Catcher in the Rye' added.


C:\Users\Chandan D\AppData\Local\Temp\ipykernel_22420\986343101.py:20: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


Book '1984' added.
Book 'To Kill a Mockingbird' added.
Book 'Brave New World' added.
Author 'J.D. Salinger' added.


C:\Users\Chandan D\AppData\Local\Temp\ipykernel_22420\986343101.py:29: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


Author 'George Orwell' added.
Author 'Harper Lee' added.
Author 'Aldous Huxley' added.


C:\Users\Chandan D\AppData\Local\Temp\ipykernel_22420\986343101.py:38: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


Genre 'Literature' added.
Genre 'Dystopian' added.
Genre 'Classic' added.


C:\Users\Chandan D\AppData\Local\Temp\ipykernel_22420\986343101.py:47: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


User 'Alice' likes the book 'The Catcher in the Rye'.
User 'Bob' likes the book '1984'.
User 'Carol' likes the book 'To Kill a Mockingbird'.
Author 'J.D. Salinger' wrote the book 'The Catcher in the Rye'.
Author 'George Orwell' wrote the book '1984'.
Author 'Harper Lee' wrote the book 'To Kill a Mockingbird'.
Author 'Aldous Huxley' wrote the book 'Brave New World'.
Book 'The Catcher in the Rye' belongs to genre 'Literature'.
Book '1984' belongs to genre 'Dystopian'.
Book 'To Kill a Mockingbird' belongs to genre 'Classic'.
Book 'Brave New World' belongs to genre 'Dystopian'.
Books liked by Alice: ['The Catcher in the Rye']
Books recommended for Alice (by genre): []
Books recommended for Alice (by author): []
